In [1]:
import numpy as np
import re
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from tensorflow import device
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14072087133445680968
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6686252073
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8819084240028472863
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:1c:00.0, compute capability: 6.1"
]


In [ ]:
f = open("KakaoTalkChats.txt", 'r', encoding='UTF8')
lines = f.readlines()
# 불러온 대화 내역의 emoji를 제거해주는 부분입니다.
# refer from https://stackoverflow.com/a/49146722/330558
emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                           "]+", flags=re.UNICODE)
lines = [emoji_pattern.sub(r'', k) for k in lines]

In [3]:
#회원님 이라는 단어를 찾으면 그 앞 문장과 현재 문장을 질문-대답으로 묶어냅니다.
data = []
for i in range(len(lines)):
    if '회원님' in lines[i]:
        data.append([lines[i - 1], lines[i]])
#묶어낸 대화를 ' : '를 기준으로 잘라줍니다.(이름을 제거하고 대화만 남깁니다)
for i in range(len(data)):
    text, reply = data[i]
    text = text[text.find(" : ") + 3 :]
    reply = reply[reply.find(" : ") + 3 :]
    data[i] = [text, reply]
#딥러닝 모델에 들어갈 입력의 크기를 정의해줍니다.
TEXT_LEN = 30
#string의 길이를 늘리거나 줄여주는 함수입니다. 짧으면 왼쪽에 공백 추가, 길면 자릅니다.
def max_text(string, t_len):
    if len(string) >= t_len: return string[-TEXT_LEN :]
    else: return (' ' * (t_len - len(string))) + string

#text-reply 형식의 대화를 recurrent한 데이터로 바꿔줍니다.
rawX = []
rawY = []
for i in range(len(data)):
    for j in range(len(data[i][1])):
        rawX.append(max_text(data[i][0][j:] + data[i][1][:j], TEXT_LEN))
        rawY.append(data[i][1][j])

In [4]:
#글자를 숫자로 바꿔주기 위한 dictionarly를 만들어줍니다.
flat_data = np.array(data).reshape(-1)
flat_data = ''.join(flat_data)
chars = sorted(set(list(flat_data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
#데이터를 숫자로 바꿔줍니다.
X_ = []
Y_ = []
for i in range(len(rawX)):
    cache = [char_to_int[s]for s in rawX[i]]
    X_.append(cache)
    Y_.append([char_to_int[rawY[i]]])


In [6]:
#ndarray로 바꿔주고 출력은 one-hot-encoding 해줍니다.
X = np.array(X_)
X = np.reshape(X, (len(X_), TEXT_LEN, 1))
Y = np_utils.to_categorical(Y_)

In [28]:
#서술된 모델을 구성합니다. 이때 checkpoint로 epooch 중간중간의 weight를 저장하여줍니다.
with device("/GPU:0"):
    model = Sequential()
    model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.1))
    model.add(Dense(Y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

In [ ]:
#학습시킵니다.
    model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks_list)

In [29]:
#저장된 weight를 불러오고, 숫자로 된 데이터를 다시 문자로 바꿔주는 dictionary를 정의합니다.
filename = "weights-improvement-20-3.4044_size1024.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [80]:
# 입력을 넣고 출력을 관찰합니다.

input_text = max_text("테스트할 입력을 넣는 부분입니다.\n", TEXT_LEN)
input_text = [char_to_int[s]for s in input_text]
print("".join([int_to_char[value] for value in input_text]), end="")

while True:
    x = np.reshape(input_text, (1, len(input_text), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    if index == 0 : break
    result = int_to_char[index]
    seq_in = [int_to_char[v] for v in input_text]
    sys.stdout.write(result)
    input_text.append(index)
    input_text = input_text[1:len(input_text)]

                           ㅇㅈ
나영 너가 이리 얼주 나 참 사이 있 있 있 있는 방 있는 분고 있는 듯 잘지 않을 것 같아

In [52]:
with device("/GPU:0"):
    model2 = Sequential()
    model2.add(LSTM(500, input_shape=(X.shape[1], X.shape[2])))
    model2.add(Dropout(0.1))
    model2.add(Dense(Y.shape[1], activation='softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='adam')
filename = "weights-improvement-50-3.2271.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')

In [104]:
coinput = "테스트할 입력을 여기에 적으세요\n"
input_text = max_text(coinput, TEXT_LEN)
input_text = [char_to_int[s]for s in input_text]
print("입력 :","".join([int_to_char[value] for value in input_text]), end="")
print("출력 :", end="")
while True:
    x = np.reshape(input_text, (1, len(input_text), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    if index == 0 : break
    result = int_to_char[index]
    seq_in = [int_to_char[v] for v in input_text]
    sys.stdout.write(result)
    input_text.append(index)
    input_text = input_text[1:len(input_text)]
print("")

input_text = max_text(coinput, TEXT_LEN)
input_text = [char_to_int[s]for s in input_text]
print("입력 :","".join([int_to_char[value] for value in input_text]), end="")
print("출력 :", end="")
while True:
    x = np.reshape(input_text, (1, len(input_text), 1))
    prediction = model2.predict(x, verbose=0)
    index = np.argmax(prediction)
    if index == 0 : break
    result = int_to_char[index]
    seq_in = [int_to_char[v] for v in input_text]
    sys.stdout.write(result)
    input_text.append(index)
    input_text = input_text[1:len(input_text)]

입력 :      술 많이 먹은 것 같은데 집은 잘 들어갔어?
출력 :응선 그거 제 이는 있 이는 있 부 있는 동 이지 것 같아
입력 :      술 많이 먹은 것 같은데 집은 잘 들어갔어?
출력 :일선 가능하 수 있는 것 같은